<a href="https://colab.research.google.com/github/judeha/addnoise/blob/main/Add_noise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Drive setup
from google.colab import drive, files
drive.mount('/content/drive/')

path="drive/My Drive/Dell Lab/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Import dependencies
import numpy as np
import pandas as pd
import math
import json
import os

In [ ]:
# Import datasets from hugging face
!pip install datasets

In [ ]:
from datasets import load_dataset, Dataset
raw_datasets=load_dataset("conll2003")

In [ ]:
from copy import deepcopy
conll=deepcopy(raw_datasets)

In [ ]:
# Import error data from drive | not necessary anymore 
fp = path + 'error_counts.npy'

counts = np.load(fp)

a_subs = counts[ord('a')]

for i, x in enumerate(a_subs):
    if x != 0:
        print('{}: {}'.format(chr(i), x))

In [ ]:
# Import homoglyph list | not necessary anymore
f = open(path + "homoglyph_list.json")

homoglyph = pd.read_json(f)
homoglyph = homoglyph.sort_values(by=["a","b"])
print(homoglyph)

In [ ]:
# Combine tokens into text
from nltk.tokenize.treebank import TreebankWordDetokenizer
corpus = conll['train']['tokens']

for i in range(len(corpus)):
  corpus[i] = TreebankWordDetokenizer().detokenize(corpus[i])

print(corpus)

In [ ]:
# Import and reshape homoglyph list
lst = pd.read_csv(path + 'homoglyph.csv')
lst = lst.drop('Unnamed: 0', axis=1)

df = pd.DataFrame(lst)
homoglyph = pd.pivot_table(df, index='a', columns='b', values='sim_score', fill_value=0)

print(homoglyph)

In [ ]:
# Calculate probability of substitution
import statistics as stats
from math import isnan

err = 0.05

# find sums of similarities 
sums = []

# NOTE: not sure why iterrows doesn't work here
for i in range(len(homoglyph.index)):
  row = homoglyph.iloc[i]
  lst = row.values.tolist()
  lst = [x for x in lst if isnan(x)==False]
  sums.append(sum(lst))

print(sums)
# standardize using max sum
mx = max(sums)

def helper(n):
  return err * (n/mx)

subst_prob = list(map(helper,sums))
print(subst_prob)

In [ ]:
# Convert back to dataframe with labels
labels = homoglyph.columns.values.tolist()
arr = [labels, subst_prob]

ref = pd.DataFrame(arr, columns=labels)
ref.fillna(0)
print(ref)

In [ ]:
# Normalize similarity scores to get CDFs
def find_max(r):
  return r.max(axis=0)

def normalize(n, factor):
  return n / factor

homoglyph = homoglyph.apply(lambda row : normalize(row, find_max(row)), axis=1)
print(homoglyph)

In [ ]:
%reset_selective -f "homoglyph"

In [ ]:
from re import S
from prompt_toolkit.layout.dummy import D
# Sort similarity scores by CDFs for each char
# Construct dict of k,v pairs

subst_cdf = {}
for r in range(len(homoglyph.index)):
  Dict = {}
  for c in range(len(homoglyph.columns)):
    Dict.update({homoglyph.iloc[r][c]: homoglyph.columns.values[c]})
  s=dict(sorted(Dict.items()))
  subst_cdf.update({homoglyph.index[r]: s})

In [ ]:
# Substitution function
def substitute(c: str):
  indicator = ref[c]
  bin = np.random.binomial(1, indicator)
  if bin:
    prob = np.random.uniform(0,1)
    # NOTE: unfinished, need to index replacement from homoglyph.index(c) that is closest to prob. let this equal replacement
    replacement = 'a'
    return replacement
  else: return c

In [ ]:
# Test on colln
for text in corpus[1:4]:
  for i in range(len(text)):
    temp = list(text)
    print(temp)
    # NOTE: conversion not working
    temp[i] = substitute(temp[i])
    text = "".join(temp)

In [ ]:
import datasets
import json
import random
from bisect import bisect_left


random.seed(42)


def substitute(char):
    if char in char_list and random.random() < scaled_sim_sums[char]:
        i = bisect_left(subst_cdfs_probs[char], random.random())
        return subst_cdfs_chars[char][i]
    else:
        return char


def insert():
    return random.choice(char_list)


def add_noise_to_word(word):
    chars = list(word)
    for i, c in enumerate(chars):
        if random.random() < 0.02:
            chars[i] = ''
            continue
        if random.random() < 0.02:
            chars[i] = substitute(c)
        if random.random() < 0.02:
            chars[i] = chars[i] + insert()
        if random.random() < 0.02:
            chars[i] = chars[i] + ' '
    return ''.join(chars)


def tokenize(string, label):
    # Split on whitespace, keep only non-empty strings
    words = list(filter(lambda w: w, string.split()))
    if words == []:
        return zip([], [])
    # Associate labels with each word, changing beginning tags into
    # inside tags for every word past the first
    labels = [label] + [label + (label % 2) for _ in range(len(words)-1)]
    return zip(words, labels)


def add_noise_to_sentence(sentence):
    # Randomly delete spaces
    noisy_sentence1  = []
    for word, label in sentence:
        if random.random() < 0.02 and noisy_sentence1:
            # Simluate space deletion by combining previous word with current word
            prev_word, prev_label = noisy_sentence1[-1]
            new_word = prev_word + word
            new_label = prev_label if prev_label != 0 else label
            noisy_sentence1[-1] = new_word, new_label
        else:
            noisy_sentence1.append((word, label))

    # Add noise to each word
    noisy_sentence2 = []
    for word, label in noisy_sentence1:
        noisy_string = add_noise_to_word(word)
        # Tokenize the noisy string into words since the noise will add spaces
        noisy_sentence2.extend(tokenize(noisy_string, label))

    return noisy_sentence2


def add_noise(data):
    return [add_noise_to_sentence(sentence) for sentence in data]


with open('homoglyph_probability_info.json') as f:
    homoglyph_info = json.load(f)

scaled_sim_sums = homoglyph_info['scaled_sim_sums']
subst_cdfs = homoglyph_info['subst_cdfs']
char_list = list(scaled_sim_sums.keys())

subst_cdfs_chars = {
    key_char: list(char_cum_probs.keys())
    for key_char, char_cum_probs in subst_cdfs.items()
}
subst_cdfs_probs = {
    key_char: list(char_cum_probs.values())
    for key_char, char_cum_probs in subst_cdfs.items()
}

conll = datasets.load_dataset("conll2003")
conll_train, conll_val, conll_test = (
    [
        list(zip(sentence_words, sentence_labels))
        for sentence_words, sentence_labels
        in zip(conll[split]['tokens'], conll[split]['ner_tags'])
    ]
    for split in ('train', 'validation', 'test')
)

conll_train_noisy = add_noise(conll_train)
conll_val_noisy = add_noise(conll_val)
conll_test_noisy = add_noise(conll_test)

id2label = {
    0: 'O',
    1: 'B-PER',
    2: 'I-PER',
    3: 'B-ORG',
    4: 'I-ORG',
    5: 'B-LOC',
    6: 'I-LOC',
    7: 'B-MISC',
    8: 'I-MISC'
}

with open('noisy_conll-2003_train.txt', 'w') as f:
    for sentence in conll_train_noisy:
        for word, label in sentence:
            f.write(f'{word} {id2label[label]}\n')
        f.write('\n')
with open('noisy_conll-2003_val.txt', 'w') as f:
    for sentence in conll_val_noisy:
        for word, label in sentence:
            f.write(f'{word} {id2label[label]}\n')
        f.write('\n')
with open('noisy_conll-2003_test.txt', 'w') as f:
    for sentence in conll_test_noisy:
        for word, label in sentence:
            f.write(f'{word} {id2label[label]}\n')
        f.write('\n')